# Deep Convolutional GAN (DC-GAN)

Originally proposed by [Radford et al.](https://arxiv.org/pdf/1511.06434.pdf) is their work titled Unsupervised Representation Learning With Deep Convolutions Generative Adversarial Networks. This network uses a basic implementation where generator and discriminator models use convolutional layers, batch normalization and Upsampling. This notebook trains both networks using ADAM optimizer to play the minimax game. We showcase the effectiveness using MNIST digit generation

## Load Libraries

In [1]:
import os
import numpy as np
import math

import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision import datasets
from torch.autograd import Variable
from torch.utils.data import DataLoader

from torchvision.utils import save_image
import torchvision.transforms as transforms

## Check GPU

In [2]:
CUDA = True if torch.cuda.is_available() else False

## Set Parameters

In [3]:
NUM_CHANNELS = 1
IMG_DIM = 32
BATCH_SIZE = 64
Z_DIM = 256 # Noise Vector Dimension
N_EPOCHS = 100
SAMPLE_INTERVAL = 400
IMG_SHAPE = (NUM_CHANNELS,IMG_DIM, IMG_DIM)

## Get MNIST Dataset

In [4]:
# create directory
os.makedirs("images", exist_ok=True)

# download dataset
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "data/mnist",
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(IMG_DIM), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ),
    ),
    batch_size=BATCH_SIZE,
    shuffle=True,
)

100%|██████████| 9912422/9912422 [00:00<00:00, 91008555.08it/s]


Extracting data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to data/mnist/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 96599436.86it/s]


Extracting data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to data/mnist/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 30783127.75it/s]


Extracting data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to data/mnist/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 5945857.92it/s]


Extracting data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/mnist/MNIST/raw



## Discriminator Model

In [5]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        def discriminator_block(input_filters, output_filters, bn=True):
            block = [
                nn.Conv2d(input_filters, output_filters, 3, 2, 1),
                nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0.25),
                nn.BatchNorm2d(output_filters, 0.8)
                     ]
            return block

        self.model = nn.Sequential(
            *discriminator_block(NUM_CHANNELS, 16),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
        )

        # Set params for downsampled image
        ds_size = IMG_DIM // 2 ** 4
        self.adv_layer = nn.Sequential(nn.Linear(128 * ds_size ** 2, 1), nn.Sigmoid())

    def forward(self, img):
        out = self.model(img)
        out = out.view(out.shape[0], -1)
        validity = self.adv_layer(out)

        return validity

## Generator Model

In [6]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.init_size = IMG_DIM // 4
        self.l1 = nn.Sequential(nn.Linear(Z_DIM, 128 * self.init_size ** 2))

        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, NUM_CHANNELS, 3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, z):
        out = self.l1(z)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img


## Attach Loss & Optimizers

In [7]:
# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

# Loss function
adversarial_loss = torch.nn.BCELoss()

# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

In [8]:
if CUDA:
  generator.cuda()
  discriminator.cuda()
  adversarial_loss.cuda()

  Tensor = torch.cuda.FloatTensor
else:
  Tensor = torch.FloatTensor

## Train DC-GAN

In [9]:
for epoch in range(N_EPOCHS):
    for i, (imgs, _) in enumerate(dataloader):

        # Set Real and Fake Labels
        valid = Variable(Tensor(imgs.size(0), 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.size(0), 1).fill_(0.0), requires_grad=False)

        # Set Variable for real images
        real_imgs = Variable(imgs.type(Tensor))

        #  Train Generator
        optimizer_G.zero_grad()

        # Sample noise vector z for generator
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], Z_DIM))))

        # get generator output
        gen_imgs = generator(z)

        # Calculate and update generator loss
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)
        g_loss.backward()
        optimizer_G.step()

        #  Train Discriminator
        optimizer_D.zero_grad()

        # Calculate Discriminator loss over Fake and Real Samples
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        # Update Discriminator loss
        d_loss.backward()
        optimizer_D.step()
        print(f'Epoch: {epoch}/{N_EPOCHS}-Batch: {i}/{len(dataloader)}--D.loss:{d_loss.item():.4f},G.loss:{g_loss.item():.4f}')

        batches_done = epoch * len(dataloader) + i
        if batches_done % SAMPLE_INTERVAL == 0:
            save_image(gen_imgs.data[:25], f"images/{batches_done}.png"  , nrow=5, normalize=True)

<ipython-input-9-3ccabbd564d7>:5: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:83.)
  valid = Variable(Tensor(imgs.size(0), 1).fill_(1.0), requires_grad=False)


Streaming output truncated to the last 5000 lines.
Epoch: 37/100-Batch: 551/938--D.loss:0.1575,G.loss:0.1577
Epoch: 37/100-Batch: 552/938--D.loss:0.9384,G.loss:2.2412
Epoch: 37/100-Batch: 553/938--D.loss:0.1065,G.loss:4.1597
Epoch: 37/100-Batch: 554/938--D.loss:0.4071,G.loss:5.9810
Epoch: 37/100-Batch: 555/938--D.loss:0.1373,G.loss:2.4760
Epoch: 37/100-Batch: 556/938--D.loss:0.1125,G.loss:2.9385
Epoch: 37/100-Batch: 557/938--D.loss:0.2515,G.loss:1.1463
Epoch: 37/100-Batch: 558/938--D.loss:0.3536,G.loss:0.9112
Epoch: 37/100-Batch: 559/938--D.loss:0.4207,G.loss:0.4566
Epoch: 37/100-Batch: 560/938--D.loss:0.2358,G.loss:0.5702
Epoch: 37/100-Batch: 561/938--D.loss:0.1209,G.loss:4.8794
Epoch: 37/100-Batch: 562/938--D.loss:0.5767,G.loss:2.0759
Epoch: 37/100-Batch: 563/938--D.loss:0.1736,G.loss:2.7724
Epoch: 37/100-Batch: 564/938--D.loss:0.2890,G.loss:0.5522
Epoch: 37/100-Batch: 565/938--D.loss:0.5926,G.loss:0.7566
Epoch: 37/100-Batch: 566/938--D.loss:0.1875,G.loss:2.4602
Epoch: 37/100-Batch: 

KeyboardInterrupt: 